In [ ]:
import pandas as pd
import numpy as np
import nibabel as nib
import os
import scipy.stats as scp
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
import itertools
from scipy.signal import hilbert
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import scipy.signal as scs
import json
import pickle
import plotly.graph_objects as go
from tqdm.auto import tqdm
from itertools import combinations
import statsmodels.formula.api as smf
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

sns.set(context='talk', style='white', font='Arial')

today = date.today().strftime('%Y%m%d')

project_dir = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/'
data_dir = project_dir + 'proc/group/parcel_timeseries/sub_ts/'
out_dir = project_dir + 'proc/clin/'
os.makedirs(out_dir,exist_ok=True)

big_data_dir = '/Users/catcamacho/Documents/bigdata/hbn_clin/'

sample_file = project_dir + 'proc/group/datasets_info/sample_gord.32k_fs_LR.pscalar.nii'
atlas_file = project_dir + 'proc/null_lL_WG33/Gordon333_SeitzmanSubcortical.32k_fs_LR.dlabel.nii'

ax0 = nib.load(sample_file).header.get_axis(0)
ax1 = nib.load(sample_file).header.get_axis(1)

TR = 0.8

# get network labels
parcel_labels = nib.load(sample_file).header.get_axis(1).name
network_labels = []
for s in parcel_labels:
    b = s.split('_')
    if len(b)<2:
        network_labels.append(b[0])
    else:
        network_labels.append(b[1])
network_labels = np.array(network_labels)
network_names, network_sizes = np.unique(network_labels, return_counts=True)

# load timeseries data info
subinfo = pd.read_csv(project_dir + 'proc/group/datasets/firstleveldatalabels_withpub_thresh0.8_20220412.csv', index_col=0)
mfqsr = pd.read_csv(os.path.join(out_dir, 'MFQsr_factorscores_20220629.csv'), index_col=0)
mfqsr.index = ['sub-{0}'.format(s) for s in mfqsr.index]
mfqsr.index.name='sub'
mfqpr = pd.read_csv(os.path.join(out_dir, 'MFQpr_factorscores_20220629.csv'), index_col=0)
mfqpr.index = ['sub-{0}'.format(s) for s in mfqpr.index]
mfqpr.index.name='sub'

scaredsr = pd.read_csv(os.path.join(project_dir, 'phenotypic_data','9994_SCARED_SR_20210322.csv'), index_col='EID', skiprows=[1]).loc[:,['SCARED_SR_SC','SCARED_SR_GD','SCARED_SR_SP']]
scaredsr.index = ['sub-{0}'.format(a) for a in scaredsr.index]
scaredsr.index.name = 'sub'

scaredpr = pd.read_csv(os.path.join(project_dir, 'phenotypic_data','9994_SCARED_P_20210322.csv'), index_col='EID', skiprows=[1]).loc[:,['SCARED_P_SC','SCARED_P_GD','SCARED_P_SP']]
scaredpr.index = ['sub-{0}'.format(a) for a in scaredpr.index]
scaredpr.index.name = 'sub'

adhd = pd.read_csv(os.path.join(project_dir, 'phenotypic_data','9994_SWAN_20210322.csv'), index_col='EID', skiprows=[1]).loc[:,'SWAN_Avg']
adhd.index = ['sub-{0}'.format(a) for a in adhd.index]
adhd.index.name = 'sub'

clininfo = pd.read_csv(os.path.join(out_dir, 'depanx_scores_preproc_20220519.csv'), low_memory=False, index_col=0).drop(['age'],axis=1)
subinfo = subinfo.merge(clininfo, how='left', left_index=True, right_index=True)
subinfo = subinfo.merge(mfqpr, how='left', left_index=True, right_index=True)
subinfo = subinfo.merge(mfqsr, how='left', left_index=True, right_index=True)
subinfo = subinfo.merge(adhd, how='left', left_index=True, right_index=True)
subinfo = subinfo.drop(['set','sub','cond','SCARED_P_SC','SCARED_SR_SC'], axis=1)
subinfo = subinfo.drop_duplicates()
subinfo.index.name='sub'

subinfo = subinfo.merge(scaredsr, how='left', left_index=True, right_index=True)
subinfo = subinfo.merge(scaredpr, how='left', left_index=True, right_index=True)

# assign developmental groups
subinfo['age_group'] = 'younger'
subinfo.loc[(subinfo['age']>10), 'age_group'] = 'older'

subinfo = subinfo.loc[(np.isfinite(subinfo['SCARED_SR_SC']) | np.isfinite(subinfo['SCARED_P_SC'])), :]

## set up functions

In [ ]:
def compile_ts_data(subdf, movie, datadir, outfile):
    """
    combine data for each movie together into 1 file
    
    Parameters
    ----------
    subdf: DataFrame
        A dataframe with subject IDs as the index. Includes IDs for all usable data.
    movie: str
        Corresponds with the str for the movie content to concatenate (e.g., "DM" or "TP").
    datadir: folder path
        Path to folder with the subject timeseries ciftis.
    outfile: file path
        Path including filename to save the output data of shape Ntimepoints x Nparcels x Nsubjects.
    
    Returns
    -------
    data: numpy array
        The compiled data of shape Ntimepoints x Nparcels x Nsubjects
    """
    if not isinstance(subdf, pd.DataFrame):
        subdf = pd.read_csv(subdf, index_col=0)
    
    for sub in subdf.index:
        file = '{0}{1}_task-movie{2}_bold1_AP_Atlas_rescale_resid0.9_filt_gordonseitzman.32k_fs_LR.ptseries.nii'.format(datadir,sub, movie)
        if sub == subdf.index[0]:
            data = StandardScaler().fit_transform(nib.load(file).get_fdata())
            data = np.expand_dims(data, axis=2)
        else:
            t = StandardScaler().fit_transform(nib.load(file).get_fdata())
            t = np.expand_dims(t, axis=2)
            data = np.concatenate([data,t],axis=2)
    
    print('Compile data from {0} brain regions measured at {1} timepoints from {2} participants.'.format(data.shape[1],data.shape[0],data.shape[2]))
    np.save(outfile, data)
    return(data)


def compute_group_phase(group_ts_data, outfile):
    """
    convert parcel ts to standard units & compute phase angles for each parcel ts
    
    Parameters
    ----------
    group_ts_data: filepath OR numpy array
        File or numpy array with compiled timeseries data of shape Ntimepoints x Nparcels x Nsubjects 
    
    Returns
    -------
    group_phase_data: numpy array
        The compiled data of shape Ntimepoints x Nparcels x Nsubjects
    """
    
    if not isinstance(group_ts_data, np.ndarray):
        group_ts_data = np.load(group_ts_data)
    
    group_phase_data = np.zeros_like(group_ts_data)
    
    for a in range(0,group_ts_data.shape[1]):
        for b in range(0,group_ts_data.shape[2]):
            group_phase_data[:,a,b] = np.angle(hilbert(group_ts_data[:,a,b]), deg=False)
    
    np.save(outfile, group_phase_data)
    
    return(group_phase_data)

In [ ]:
def compute_isps(group_phase_data, outprefix, savemean=True, small=False):
    """
    parcel-wise inter-subject phase synchrony- output pairwise IPS and mean global IPS
    
    Parameters
    ----------
    group_phase_data: numpy array
        The compiled data of shape Ntimepoints x Nparcels x Nsubjects
        
    Returns
    -------
    isps_data: numpy array
        intersubject phase synchrony data of shape Nparcels x Nsubjects x Nsubjects x Ntimepoints
    mean_isps_data: numpy array
        intersubject phase synchrony data, averaged across time, of shape Nparcels x Nsubjects x Nsubjects
        
    """
    if not isinstance(group_phase_data, np.ndarray):
        group_phase_data = np.load(group_phase_data)
    
    if os.path.isdir(outprefix):
        file_name = os.path.join(outprefix, 'isps_data.dat')
    else:
        file_name = outprefix + 'isps_data.dat'
        
    if not small:
        isps_data = np.memmap(file_name, dtype=np.float32, mode='w+',
                              shape=(group_phase_data.shape[1],group_phase_data.shape[2],
                                     group_phase_data.shape[2],
                                     group_phase_data.shape[0]))
    else:
        isps_data = np.empty((group_phase_data.shape[1],group_phase_data.shape[2],
                              group_phase_data.shape[2],group_phase_data.shape[0]))
    
    subs = range(0, group_phase_data.shape[2])
    for region in range(0, group_phase_data.shape[1]):
        combs = itertools.combinations(subs, 2)
        for c in combs:
            sub1 = group_phase_data[:, region, c[0]]
            sub2 = group_phase_data[:, region, c[1]]
            a = 1 - np.sin(np.abs(sub1 - sub2) / 2)
            isps_data[region,c[0],c[1],:] = a
            isps_data[region,c[1],c[0],:] = a
        
    if small:
        np.save(file_name, isps_data)
    if savemean:
        mask = np.tri(isps_data.shape[2], isps_data.shape[2], -1, dtype=int)
        mean_isps_data = np.mean(isps_data[:,mask==1,:], axis=1)
        if os.path.isdir(outprefix):
            mean_file_name = os.path.join(outprefix, 'mean_isps_data.npy')
        else:
            mean_file_name = outprefix + 'mean_isps_data.npy'
        
        np.save(mean_file_name, mean_isps_data.T)
    
        return(mean_isps_data, isps_data)
    else:
        return(isps_data)


def intersubject_timeseries_correlation(data, outprefix, ax0=ax0, ax1=ax1):
    """
    Parameters
    ----------
    data: numpy array
        data in the shape of Ntimepoints x Nregions x Nsubjects
    outprefix: str
        name to save ISC data to
    
    Returns
    -------
    intersub_isc: numpy array
        intersubject spearman correlations in the shape of Nregions x Nsubjects x Nsubjects
    group_isc: numpy array
        group mean spearman correlations in the shape of Nregions
    """
    subs = range(0,data.shape[2])
    
    intersub_isc = np.zeros((data.shape[1],data.shape[2],data.shape[2]))
    group_isc = np.zeros((data.shape[1]))
    mask = np.tri(data.shape[2], data.shape[2], -1, dtype=int)
    
    for r in range(0, data.shape[1]):
        intersub_isc[r, :, :]= np.corrcoef(data[:, r, :], rowvar=False)
            
    for r in range(0, data.shape[1]):
        group_isc[r] = np.mean(intersub_isc[r,:,:][mask==1])
    
    np.save(outprefix + 'intersub_timeseries_ISC.npy', intersub_isc)
    img = nib.cifti2.cifti2.Cifti2Image(np.expand_dims(group_isc, axis=0), (ax0, ax1))
    nib.save(img, outprefix + 'mean_timseries_ISC.pscalar.nii')
    
    return(intersub_isc, group_isc)


def intersubject_distance(data, outfile_prefix):
    """
    Compute static pairwise intersubject similarity
    
    Parameters
    ----------
    data: numpy array
        1D array of subject data (i.e., each participant contributes exactly 1 measure)
    outfilename: str
        name to save distance data to
    
    Returns
    -------
    isdistances: numpy array
        intersubject distances in the shape of Nsubjects x Nsubjects x Nmetrics
    """
    subs = range(0,data.shape[0])


    # NN
    nn = np.zeros((data.shape[0],data.shape[0]))
    combs = itertools.combinations(subs, 2)
    for c in combs:
        nn[c[0],c[1]] = np.max(data) - abs(data[c[0]] - data[c[1]])
        nn[c[1],c[0]] = np.max(data) - abs(data[c[0]] - data[c[1]])
    np.save(outfile_prefix + '_NN.npy', nn)

    # AnnaK mean
    annakmean = np.zeros((data.shape[0],data.shape[0]))
    combs = itertools.combinations(subs, 2)
    for c in combs:
        annakmean[c[0],c[1]] = (data[c[0]] + data[c[1]]) / 2
        annakmean[c[1],c[0]] = (data[c[0]] + data[c[1]]) / 2
    np.save(outfile_prefix + '_annakmean.npy', annakmean)
    
    # AnnaK max min mean
    AnnaKmaxminmean = np.zeros((data.shape[0],data.shape[0]))
    combs = itertools.combinations(subs, 2)
    for c in combs:
        AnnaKmaxminmean[c[0],c[1]] = np.max(data) - ((data[c[0]] + data[c[1]]) / 2)
        AnnaKmaxminmean[c[1],c[0]] = np.max(data) - ((data[c[0]] + data[c[1]]) / 2)
    np.save(outfile_prefix + '_annakmaxminmean.npy', AnnaKmaxminmean)

    # AnnaK min
    annakmin = np.zeros((data.shape[0],data.shape[0]))
    combs = itertools.combinations(subs, 2)
    for c in combs:
        annakmin[c[0],c[1]] = min([data[c[0]],data[c[1]]])
        annakmin[c[1],c[0]] = min([data[c[0]],data[c[1]]])
    np.save(outfile_prefix + '_annakmin.npy', annakmin)

    # AnnaK max minus min
    annakmaxminmax = np.zeros((data.shape[0],data.shape[0]))
    combs = itertools.combinations(subs, 2)
    for c in combs:
        annakmaxminmax[c[0],c[1]] =np.max(data) -  max([data[c[0]],data[c[1]]])
        annakmaxminmax[c[1],c[0]] = np.max(data) - max([data[c[0]],data[c[1]]])
    np.save(outfile_prefix + '_annakmaxminmax.npy', annakmaxminmax)
        
    # AnnaK absmean
    annakabsmean = np.zeros((data.shape[0],data.shape[0]))
    combs = itertools.combinations(subs, 2)
    for c in combs:
        annakabsmean[c[0],c[1]] = abs(data[c[0]] - data[c[1]]) * ((data[c[0]] + data[c[1]]) / 2)
        annakabsmean[c[1],c[0]] = abs(data[c[0]] - data[c[1]]) * ((data[c[0]] + data[c[1]]) / 2)
    np.save(outfile_prefix + '_annakabsmean.npy', annakabsmean)
    
    isdistances = {'NN': nn, 
                   'AnnaKmean': annakmean, 
                   'AnnaKmin': annakmin, 
                   'AnnaKabsmean': annakabsmean, 
                   'AnnaKmaxminmean': AnnaKmaxminmean, 
                   'AnnaKmaxminmax': annakmaxminmax}
    return(isdistances)

In [ ]:
def static_brain_bx_isrsa(brain_sim_data, bx_sim_data, outfilename=None):
    """
    
    Parameters
    ----------
    brain_sim_data: numpy ndarray
        Data in the shape of Nsubjects x Nsubjects
    bx_sim_data: numpy ndarray
        Data in the shape of Nsubjects x Nsubjects
        
    Returns
    -------
    rsa_report: pandas DataFrame
        Pandas DataFrame with inter-subject representational similarity statistics
    """
    rsa_report = pd.DataFrame(columns=['SpearR','SpearPvalue'])
    
    mask = np.tri(bx_sim_data.shape[0], bx_sim_data.shape[0], -1, dtype=int)
    bx_sim = bx_sim_data[mask==1]
    brain_sim = brain_sim_data[mask==1]
    
    r, p = scp.spearmanr(bx_sim, brain_sim)
    rsa_report.loc[0,'SpearR'] = r
    rsa_report.loc[0,'SpearPvalue'] = p
    if outfilename:
        sns.scatterplot(bx_sim, brain_sim)
        plt.title('Similarity Correlation')
        plt.tight_layout()
        plt.savefig(outfilename)
        plt.show()
        plt.close()
    
    return(rsa_report)

def regional_perm_bx_isrsa(regional_sim_data, bx_sim_data, outprefix, alpha=0.05, n_perms=1000, ax0=ax0, ax1=ax1):
    """
    
    Parameters
    ----------
    regional_sim_data: numpy ndarray
        Data in the shape of Nregions x Nsubjects x Nsubjects
    bx_sim_data: numpy ndarray
        Data in the shape of Nsubjects x Nsubjects
        
    Returns
    -------
    region_isrsa: numpy ndarray
        Data in the shape of Nregions
    """
    
    mask = np.tri(bx_sim_data.shape[1], bx_sim_data.shape[1], -1, dtype=int)

    # flatten behavior lower triangle
    bx_sim = bx_sim_data[mask==1]

    region_isrsa = np.zeros((regional_sim_data.shape[0]))

    for region in range(0, regional_sim_data.shape[0]):
            brain_sim = regional_sim_data[region,:,:][mask==1]
            r, p = scp.spearmanr(bx_sim, brain_sim)
            region_isrsa[region] = r

    shuff_bx = bx_sim
    perm_isrsa_null = np.zeros((n_perms, regional_sim_data.shape[0]))

    # make null distributions for each TR and region
    for a in range(0,n_perms):
        np.random.shuffle(shuff_bx)
        for region in range(0,regional_sim_data.shape[0]):
            brain_sim = regional_sim_data[region,:,:][mask==1]
            r, p = scp.spearmanr(shuff_bx, brain_sim)
            perm_isrsa_null[a, region] = r

    # compute permuted P threshold per region/TR
    raw_pvals = np.zeros(region_isrsa.shape)
    flat_null = perm_isrsa_null.flatten()
    for i, a in enumerate(region_isrsa):
        raw_pvals[i] = (np.sum((flat_null>=a).astype(int)) + 1) / (flat_null.shape[0] + 1)
        
    # save ciftis with raw values
    img = nib.cifti2.cifti2.Cifti2Image(np.expand_dims(raw_pvals, axis=0), (ax0, ax1))
    nib.save(img, outprefix + '_permsim_raw_pval.pscalar.nii')
    
    img = nib.cifti2.cifti2.Cifti2Image(np.expand_dims(region_isrsa, axis=0), (ax0, ax1))
    nib.save(img, outprefix + '_permsim_raw_rho.pscalar.nii')
    
    
    # save cifti with significant rhos only
    thresh_mask = raw_pvals<alpha

    # pvals
    thresh_pval = raw_pvals
    thresh_pval[thresh_mask==0] = np.nan
    img = nib.cifti2.cifti2.Cifti2Image(np.expand_dims(thresh_pval, axis=0), (ax0, ax1))
    nib.save(img, outprefix + '_permsim_masked_pval{0}.pscalar.nii'.format(alpha))

    # rhos
    thresh_isrsa = region_isrsa
    thresh_isrsa[thresh_mask==0] = np.nan
    thresh_isrsa[thresh_isrsa<0] = np.nan
    img = nib.cifti2.cifti2.Cifti2Image(np.expand_dims(thresh_isrsa, axis=0), (ax0, ax1))
    nib.save(img, outprefix + '_permsim_masked_rho{0}.pscalar.nii'.format(alpha))
    return(thresh_isrsa)


# compute dynamic similarity
def dynamic_brain_bx_isrsa(time_region_sim_data, bx_sim_data, outprefix, ax1=ax1):
    """
    
    Parameters
    ----------
    time_region_sim_data: numpy ndarray
        Data in the shape of Nregions x Nsubjects x Nsubjects x Ntimepoints
    bx_sim_data: numpy ndarray
        Data in the shape of Nsubjects x Nsubjects
        
    Returns
    -------
    dynamic_isrsa: pandas DataFrame
        Pandas DataFrame with inter-subject representational similarity statistics
    """
    mask = np.tri(bx_sim_data.shape[1], bx_sim_data.shape[1], -1, dtype=int)

    # flatten behavior lower triangle
    bx_sim = bx_sim_data[mask==1]

    dynamic_isrsa = np.zeros((isps_data.shape[3],isps_data.shape[0]))

    for region in range(0, isps_data.shape[0]):
        for tr in range(0, isps_data.shape[3]):
            brain_sim = isps_data[region,:,:,tr][mask==1]
            r, p = scp.spearmanr(bx_sim, brain_sim)
            dynamic_isrsa[tr, region] = r
            
    np.save(outfile_prefix + '_dynamicsim.npy', dynamic_isrsa)
    
    ax0 = nib.cifti2.cifti2_axes.SeriesAxis(0,0.8,dynamic_isrsa.shape[0], unit='second')
    img = nib.cifti2.cifti2.Cifti2Image(dynamic_isrsa, (ax0, ax1))
    nib.save(img, outfile_prefix + '_dynamicsim.ptseries.nii')

    return(dynamic_isrsa)


def perm_sig_dynamic_isrsa(time_region_sim_data, bx_sim_data, outprefix, n_perms=30, ax1=ax1, alpha=0.05, TR=0.8):
    """
    
    
    """
    
    dynamic_isrsa = dynamic_brain_bx_isrsa(time_region_sim_data, bx_sim_data, outprefix)
    
    if os.path.isfile(os.path.join(outprefix + '_perm_isrsa_null.dat')):
        perm_isrsa_null = np.memmap(os.path.join(outprefix + '_perm_isrsa_null.dat'), dtype=np.float32, mode='r',
                                    shape=(n_perms, isps_data.shape[3], isps_data.shape[0]))
    else:
        perm_isrsa_null = np.memmap(os.path.join(outprefix + '_perm_isrsa_null.dat'), dtype=np.float32, mode='w+',
                                    shape=(n_perms, isps_data.shape[3], isps_data.shape[0]))
        shuff_bx = bx_sim_data

        # make null distributions for each TR and region
        for a in range(0,n_perms):
            np.random.shuffle(shuff_bx)
            perm_isrsa_null[a, :, :] = dynamic_brain_bx_isrsa(time_region_sim_data, shuff_bx, outprefix)
    
    # compute permuted P threshold per region/TR
    orig_shape = dynamic_isrsa.shape
    flat_dyn_isrsa = dynamic_isrsa.flatten()
    raw_pvals = np.zeros(flat_dyn_isrsa.shape)
    flat_null = perm_isrsa_null.flatten()
    for i, a in enumerate(flat_dyn_isrsa):
        raw_pvals[i] = (np.sum((flat_null>=a).astype(int)) + 1) / (flat_null.shape[0] + 1)

    raw_pvals = np.reshape(raw_pvals, orig_shape)
    
    # save cifti
    ax0 = nib.cifti2.cifti2_axes.SeriesAxis(0,TR,raw_pvals.shape[0], unit='second')
    img = nib.cifti2.cifti2.Cifti2Image(dynamic_isrsa, (ax0, ax1))
    nib.save(img, outprefix + '_permdynamicsim_raw_rho.ptseries.nii')
    img = nib.cifti2.cifti2.Cifti2Image(raw_pvals, (ax0, ax1))
    nib.save(img, outprefix + '_permdynamicsim_raw_pval.ptseries.nii')
    
    # save cifti with significant rhos only
    thresh_mask = raw_pvals<alpha

    # pvals
    thresh_pval = raw_pvals
    thresh_pval[thresh_mask==0] = np.nan
    img = nib.cifti2.cifti2.Cifti2Image(thresh_pval, (ax0, ax1))
    nib.save(img, outprefix + '_permdynamicsim_thresh_pval{0}.ptseries.nii'.format(alpha))
    # rhos
    thresh_isrsa = dynamic_isrsa
    thresh_isrsa[thresh_mask==0] = np.nan
    thresh_isrsa[thresh_isrsa<0] = np.nan
    img = nib.cifti2.cifti2.Cifti2Image(thresh_isrsa, (ax0, ax1))
    nib.save(img, outprefix + '_permdynamicsim_thresh_rho{0}.ptseries.nii'.format(alpha))

In [ ]:
def region_isrsa_fdr(disc_rho, disc_pval, rep_rho, rep_pval, outprefix, alpha=0.05, bon_alpha=True,replace_zeros=True, ax0=ax0, ax1=ax1):
    """
    
    
    """
    
    disc_rho = nib.load(disc_rho).get_fdata()
    disc_pval = nib.load(disc_pval).get_fdata()
    rep_rho = nib.load(rep_rho).get_fdata()
    rep_pval = nib.load(rep_pval).get_fdata()
    
    if replace_zeros:
        disc_pval[disc_pval==0] = np.nan
        rep_pval[rep_pval==0] = np.nan
    
    if bon_alpha==True:
        bon_alpha = np.sqrt(alpha/disc_pval.shape[1])
    else:
        bon_alpha = alpha

    dmask = (disc_pval<bon_alpha).astype(int)
    rmask = (rep_pval<bon_alpha).astype(int)

    mask = np.zeros(dmask.shape)
    mask[(dmask==1) & (rmask==1)] = 1

    bonrho = np.empty(mask.shape)
    bonrho[mask==1] = np.add(disc_rho[mask==1],rep_rho[mask==1])/2
    bonrho[mask==0] = np.nan

    img = nib.cifti2.cifti2.Cifti2Image(bonrho, (ax0, ax1))
    nib.save(img, outprefix + '_maskedrho_fdr{0}.pscalar.nii'.format(round(bon_alpha,5)))
    
    
def dynamic_isrsa_fdr(disc_rho, rep_rho, discN, repN, video_dur, ratings_file, outprefix, TR=TR, alpha=0.05, parcel_labels=parcel_labels):
    
    # assign thresholds
    bon_alpha = np.sqrt(0.05/disc_rho.shape[1])
    discbonnmint = scp.t.ppf(1-bon_alpha, discN-1)
    discuncorrmint = scp.t.ppf(1-alpha, discN-1)
    repbonnmint = scp.t.ppf(1-bon_alpha, repN-1)
    repuncorrmint = scp.t.ppf(1-alpha, repN-1)

    # convert rho to t-stat
    disc_tstat = disc_rho/np.sqrt((1-disc_rho**2)/(discN-2))
    rep_tstat = rep_rho/np.sqrt((1-rep_rho**2)/(repN-2))

    # find sig ts that replicate
    disc_sigts =  (disc_tstat > discbonnmint).astype(int)
    rep_sigts =  (rep_tstat > repbonnmint).astype(int)

    sigts = np.zeros(disc_sigts.shape).astype(int)
    sigts[(disc_sigts==1) & (rep_sigts==1)] = 1
    if sigts.max()==0:
        print('No significant findings.')
    else:
        print('At least one region is significantly simliar.')
    
    # Plot and analysis replicating peaks
    times = np.arange(0, video_dur, TR)

    results = dict()

    ### Make plots ###
    for p, parcel in enumerate(parcel_labels):
        if np.max(sigts.T[p])==1:
            # set up figure
            fig, ax = plt.subplots(2,1,figsize=(12,6), sharex=True, sharey=True)
            # plot discovery
            dupeaks, duproperties = scs.find_peaks(disc_tstat.T[p], width=5, prominence=discuncorrmint)

            ax[0].plot(times, disc_tstat.T[p], color='k')
            for i_c, c in enumerate(duproperties['prominences']):
                ax[0].axvspan(duproperties['left_ips'][i_c]*TR, duproperties['right_ips'][i_c]*TR,
                            color='r', alpha=0.3)
            dcpeaks, dcproperties = scs.find_peaks(disc_tstat.T[p], width=5, prominence=discbonnmint)
            for i_c, c in enumerate(dcproperties['prominences']):
                ax[0].axvspan(dcproperties['left_ips'][i_c]*TR, dcproperties['right_ips'][i_c]*TR,
                            color='b', alpha=0.3)
            ax[0].set_xlim([0,video_dur])
            ax[0].set_title(parcel + ': Discovery', weight='bold')
            ax[0].set_ylabel('Similarity (t-stat)')

            # plot replication
            rupeaks, ruproperties = scs.find_peaks(rep_tstat.T[p], width=5, prominence=repuncorrmint)
            ax[1].plot(times, rep_tstat.T[p], color='k')
            for i_c, c in enumerate(ruproperties['prominences']):
                ax[1].axvspan(ruproperties['left_ips'][i_c]*TR, ruproperties['right_ips'][i_c]*TR,
                            color='r', alpha=0.3)
            rcpeaks, rcproperties = scs.find_peaks(rep_tstat.T[p], width=5, prominence=repbonnmint)
            for i_c, c in enumerate(rcproperties['prominences']):
                ax[1].axvspan(rcproperties['left_ips'][i_c]*TR, rcproperties['right_ips'][i_c]*TR,
                            color='b', alpha=0.3)
            ax[1].set_xlim([0,video_dur])
            ax[1].set_title(parcel + ': Replication', weight='bold')
            ax[1].set_ylabel('Similarity (t-stat)')
            ax[1].set_xlabel('Time (s)')
            plt.tight_layout()
            plt.savefig(outprefix + 'peak_similarity_{0}.svg'.format(parcel_labels[p]))
            plt.show()
            plt.close()

            # save results
            dcproperties = {k:v.tolist() for k,v in dcproperties.items()}
            duproperties = {k:v.tolist() for k,v in duproperties.items()}
            rcproperties = {k:v.tolist() for k,v in rcproperties.items()}
            ruproperties = {k:v.tolist() for k,v in ruproperties.items()}
            if (len(ruproperties['left_ips'])>0) & (len(duproperties['left_ips'])>0):
                results[parcel] = {'Discovery': {'corr': dcproperties, 'uncorr': duproperties}, 
                                   'Replication': {'corr': rcproperties, 'uncorr': ruproperties}}
            
            
    ### Characterize differences in ratings for high versus low similarity points in the video ###
    
    # load ratings and shift forward 6 TRs
    ratings = pd.read_csv(ratings_file, index_col=0)
    ratings = ratings.iloc[0:int(video_dur/TR),:]
    ratings.loc[:,ratings.columns] = MinMaxScaler().fit_transform(ratings.to_numpy())
    ratings.index = range(6,int(video_dur/TR)+6)
    ratings = pd.concat([ratings, pd.DataFrame(np.nan, index=np.arange(0,6,1), columns=ratings.columns)])
    ratings = ratings.sort_index()
    ratings = ratings.iloc[0:int(video_dur/TR),:]
    
    # characterize areas within and outside the peaks
    for parcel in results.keys():
        discmask = np.zeros(times.shape)
        for i, c in enumerate(results[parcel]['Discovery']['uncorr']['prominences']):
            start = round(results[parcel]['Discovery']['uncorr']['left_ips'][i])
            end = round(results[parcel]['Discovery']['uncorr']['right_ips'][i])
            discmask[start:end] = 1

        repmask = np.zeros(times.shape)
        for i, c in enumerate(results[parcel]['Replication']['uncorr']['prominences']):
            start = round(results[parcel]['Replication']['uncorr']['left_ips'][i])
            end = round(results[parcel]['Replication']['uncorr']['right_ips'][i])
            repmask[start:end] = 1

        tmask = np.zeros(times.shape).astype(int)
        tmask[(discmask==1) & (repmask==1)] = 1

        ratesigdiff = pd.Series(index=ratings.columns, name='sig', dtype=int)
        ratemeans = dict()
        ratestats = dict()
        if tmask.max()==1:
            for measure in ratings.columns:
                underpeak = ratings.loc[tmask==1, measure]
                outofpeak = ratings.loc[tmask==0, measure]
                underpeakmean = np.nanmean(underpeak)
                outofpeakmean = np.nanmean(outofpeak)
                rate_stat, rate_pval = scp.ttest_ind(underpeak, outofpeak, nan_policy='omit')
                ratestats[measure] = {'tstat': rate_stat, 'pval': rate_pval}
                ratesigdiff[measure] = rate_pval < alpha
                ratemeans[measure] = {'underpeak': underpeakmean, 'outofpeak': outofpeakmean}

            results[parcel]['RatingsAnalysis'] = {'MeanRatings': ratemeans, 
                                                  'Stats': ratestats}

            # plot the differences
            sigratingsnames = ratesigdiff[ratesigdiff==True].index.to_list()

            if len(sigratingsnames) < 4:
                ratings_withpeakinfo = ratings
                ratings_withpeakinfo['UnderPeak'] = tmask
                temp = pd.melt(ratings_withpeakinfo, id_vars='UnderPeak', value_vars=sigratingsnames, value_name='Level',var_name='Rating')
                plt.figure(figsize=(2+2*len(sigratingsnames),6))
                sns.barplot(x='Rating', y='Level', hue='UnderPeak', data=temp, palette=['#FFFFFF','#BCB1C2'] , linewidth=2, edgecolor='k')
                plt.tight_layout()
                plt.savefig(outprefix + 'peak_ratings_sigdiff_{0}.svg'.format(parcel))
                plt.show()
                plt.close()
            else:
                underpeak = ratings.loc[tmask==1, sigratingsnames].mean(axis=0).to_frame()
                underpeak.columns = ['mean']
                underpeak['width']=0.4
                underpeak['measure'] = underpeak.index
                outofpeak = ratings.loc[tmask==0, sigratingsnames].mean(axis=0).to_frame()
                outofpeak.columns = ['mean']
                outofpeak['width']=0.7
                outofpeak['measure'] = outofpeak.index

                fig = go.Figure()

                fig.add_trace(go.Barpolar(
                    r=outofpeak['mean'],
                    theta=outofpeak['measure'],
                    width=outofpeak['width'],
                    base=0,
                    name='OutOfPeak',
                    marker_color='#FFFFFF',
                    marker_line_color='black',
                    marker_line_width=2,
                    opacity=1,
                ))

                fig.add_trace(go.Barpolar(
                      r=underpeak['mean'],
                      theta=underpeak['measure'],
                    width=underpeak['width'],
                      name='UnderPeak',
                    base=0,
                    marker_color='#BAB3BF',
                    marker_line_color='black',
                    marker_line_width=2,
                    opacity=1,
                ))

                fig.update_layout(
                    template='plotly_white',
                  polar=dict(
                      angularaxis_tickfont_size = 14,
                    radialaxis=dict(
                      visible=True,
                      range=[0, 1]
                    )),
                  showlegend=False
                )

                fig.write_image(outprefix + 'polar_ratings_sigdiff_{0}.svg'.format(parcel))

        
    #save results
    with open(outprefix + 'full_results_corrp{0}_prom{1}_width{2}.json'.format(round(bon_alpha,3), round(discbonnmint,2), 5), 'w') as fp:
        json.dump(results, fp, indent=4)
    return(results)


def synchrony_discrep_fdr_netlevel(disc_mean_sim, disc_null, rep_mean_sim, outprefix, video_dur, alpha=0.05, 
                                   TR=TR, parcel_labels=parcel_labels, atlas_file=atlas_file):
    # assign thresholds
    bon_alpha = np.sqrt(0.05/disc_mean_sim.shape[1])

    n = int(round((len(disc_null)+1)*bon_alpha,0))
    discbonmin = disc_null[len(disc_null)-n]
    n = int(round((len(rep_null)+1)*bon_alpha,0))
    repbonmin = rep_null[len(rep_null)-n]
    if discbonmin < np.percentile(disc_mean_sim, 1-alpha):
        discbonmin = np.percentile(disc_mean_sim, 1-alpha)
        repbonmin = np.percentile(rep_mean_sim, 1-alpha)
    print("Disc sig threshold: {0}".format(discbonmin))
    print("Rep sig threshold: {0}".format(repbonmin))

    # find sig rs that replicate
    disc_sig = (disc_mean_sim > discbonmin).astype(int)
    rep_sig = (rep_mean_sim > repbonmin).astype(int)

    sigrs = np.zeros(disc_sig.shape).astype(int)
    sigrs[(disc_sig==1) & (rep_sig==1)] = 1
    
    # find parcels with at least 5 consecutive seconds of sig synchrony
    sig_parcels = np.zeros(sigrs.shape[1]).astype(int)
    sigrs_df = pd.DataFrame(sigrs, columns=parcel_labels)
    sigrs_df['time'] = range(0, len(sigrs_df))
    for i, a in enumerate(parcel_labels):
        sigrs_df['segment'] = (sigrs_df[a].diff(1) != 0).astype(int).cumsum()
        dur = pd.DataFrame({'dur': sigrs_df.groupby('segment').time.last() - sigrs_df.groupby('segment').time.first(),
                            'value': sigrs_df.groupby('segment')[a].mean()}).reset_index(drop=True)
        dur = dur.loc[dur['value']==1]
        if dur['dur'].max() > 6:
            sig_parcels[i] = 1

    # average synchrony within networks and plot
    masked_disc = disc_mean_sim
    masked_disc[:,sig_parcels==0]=np.nan
    disc_sync_df = pd.DataFrame(disc_mean_sim, columns=network_labels)
    disc_sync_df = disc_sync_df.groupby(by=network_labels, axis=1).mean().dropna(axis=1)

    masked_rep = rep_mean_sim
    masked_rep[:,sig_parcels==0]=np.nan
    rep_sync_df = pd.DataFrame(rep_mean_sim, columns=network_labels)
    rep_sync_df = rep_sync_df.groupby(by=network_labels, axis=1).mean().dropna(axis=1)

    # make figs for sig parcels sorted by network/region group
    color = (142/255, 50/255, 209/255, 1)

    ax1 = nib.load(atlas_file).header.get_axis(1)
    data = nib.load(atlas_file).get_fdata()
    ax0 = nib.load(atlas_file).header.get_axis(0)
    newmap=dict()
    newmap[0] = ax0[0][1][0]
    for net in disc_sync_df:
        parcels_keep = parcel_labels[(network_labels==net) & (sig_parcels==1)]
        for a in range(1,len(parcel_labels) +1):
            if parcel_labels[a-1] in parcels_keep:
                newmap[a] = (parcel_labels[a-1], color)
            else:
                newmap[a] = (parcel_labels[a-1], (1,1,1,0))
        ax0.label[0] = newmap
        img = nib.cifti2.cifti2.Cifti2Image(data, (ax0, ax1))
        nib.save(img, outprefix + 'sig_averaged_parcels_{0}.dlabel.nii'.format(net))
        
    # Plot and analysis replicating peaks
    times = np.arange(0, video_dur, TR)

    results = dict()

    ### Make plots ###
    for net in disc_sync_df.columns:
        fig, ax = plt.subplots(2,1,figsize=(12,6), sharex=True, sharey=True)
        # plot discovery
        ax[0].plot(times, disc_sync_df[net], color='k')
        dcpeaks, dcproperties = scs.find_peaks(disc_sync_df[net], width=5, height=discbonmin)
        for i_c, c in enumerate(dcproperties['prominences']):
            ax[0].axvspan(dcproperties['left_ips'][i_c]*TR, dcproperties['right_ips'][i_c]*TR,
                        color='#BAB3BF', alpha=0.9)
        ax[0].set_xlim([0,video_dur])
        ax[0].set_title(net + ': Discovery', weight='bold')
        ax[0].set_ylabel('Synchrony')

        # plot replication
        ax[1].plot(times, rep_sync_df[net], color='k')
        rcpeaks, rcproperties = scs.find_peaks(rep_sync_df[net], width=5, height=repbonmin)
        for i_c, c in enumerate(rcproperties['prominences']):
            ax[1].axvspan(rcproperties['left_ips'][i_c]*TR, rcproperties['right_ips'][i_c]*TR,
                        color='#BAB3BF', alpha=0.9)
        ax[1].set_xlim([0,video_dur])
        ax[1].set_title(net + ': Replication', weight='bold')
        ax[1].set_ylabel('Synchrony')
        ax[1].set_xlabel('Time (s)')
        plt.tight_layout()
        #plt.show()
        plt.savefig(outprefix + 'peak_similarity_{0}.svg'.format(net))
        plt.close()

        # save results
        dcproperties = {k:v.tolist() for k,v in dcproperties.items()}
        rcproperties = {k:v.tolist() for k,v in rcproperties.items()}
        if (len(rcproperties['left_ips'])>0) & (len(dcproperties['left_ips'])>0):
            results[net] = {'Discovery': {'corr': dcproperties},
                               'Replication': {'corr': rcproperties}}


    ### Characterize differences in ratings for high versus low similarity points in the video ###

    # load ratings and shift forward 6 TRs (4.8 seconds)
    ratings = pd.read_csv(ratings_file, index_col=0)
    ratings = ratings.iloc[0:int(video_dur/TR),:]
    ratings.loc[:,ratings.columns] = MinMaxScaler().fit_transform(ratings.to_numpy())
    ratings.index = range(6,int(video_dur/TR)+6)
    ratings = pd.concat([ratings, pd.DataFrame(np.nan, index=np.arange(0,6,1), columns=ratings.columns)])
    ratings = ratings.sort_index()
    ratings = ratings.iloc[0:int(video_dur/TR),:]

    # identify areas within and outside the peaks
    for parcel in results.keys():
        discmask = np.zeros(times.shape)
        for i, c in enumerate(results[parcel]['Discovery']['corr']['peak_heights']):
            start = round(results[parcel]['Discovery']['corr']['left_ips'][i])
            end = round(results[parcel]['Discovery']['corr']['right_ips'][i])
            discmask[start:end] = 1

        repmask = np.zeros(times.shape)
        for i, c in enumerate(results[parcel]['Replication']['corr']['peak_heights']):
            start = round(results[parcel]['Replication']['corr']['left_ips'][i])
            end = round(results[parcel]['Replication']['corr']['right_ips'][i])
            repmask[start:end] = 1

        tmask = np.zeros(times.shape).astype(int)
        tmask[(discmask==1) & (repmask==1)] = 1

        ratesigdiff = pd.Series(index=ratings.columns, name='sig', dtype=int)
        ratemeans = dict()
        ratestats = dict()
        
        if tmask.max()==1:
            for measure in ratings.columns:
                underpeak = ratings.loc[tmask==1, measure]
                outofpeak = ratings.loc[tmask==0, measure]
                underpeakmean = np.nanmean(underpeak)
                outofpeakmean = np.nanmean(outofpeak)
                rate_stat, rate_pval = scp.ttest_ind(underpeak, outofpeak, nan_policy='omit')
                ratestats[measure] = {'tstat': rate_stat, 'pval': rate_pval}
                ratesigdiff[measure] = rate_pval < alpha
                ratemeans[measure] = {'underpeak': underpeakmean, 'outofpeak': outofpeakmean}

            results[parcel]['RatingsAnalysis'] = {'MeanRatings': ratemeans, 
                                                  'Stats': ratestats}

            # plot the differences
            sigratingsnames = ratesigdiff[ratesigdiff==True].index.to_list()
            
            try:
                if len(sigratingsnames) < 4:
                    ratings_withpeakinfo = ratings
                    ratings_withpeakinfo['UnderPeak'] = tmask
                    temp = pd.melt(ratings_withpeakinfo, id_vars='UnderPeak', value_vars=sigratingsnames, value_name='Level',var_name='Rating')
                    plt.figure(figsize=(2+2*len(sigratingsnames),6))
                    fig = sns.barplot(x='Rating', y='Level', hue='UnderPeak', data=temp, palette=['#FFFFFF','#BCB1C2'] , linewidth=2, edgecolor='k')
                    plt.tight_layout()
                    plt.savefig(outprefix + 'peak_ratings_sigdiff_{0}.svg'.format(parcel))
                    #plt.show()
                    plt.close()
                    ratings = ratings.drop('UnderPeak', axis=1)
                else:
                    if 'UnderPeak' in ratings.columns:
                        ratings = ratings.drop('UnderPeak', axis=1)
                    underpeak = ratings.loc[tmask==1, sigratingsnames].mean(axis=0).to_frame()
                    if 'UnderPeak' in underpeak.index:
                        underpeak = underpeak.drop('UnderPeak', axis=0)
                    underpeak.columns = ['mean']
                    underpeak['width']=0.4
                    underpeak['measure'] = underpeak.index
                    outofpeak = ratings.loc[tmask==0, sigratingsnames].mean(axis=0).to_frame()
                    if 'UnderPeak' in outofpeak.columns:
                        outofpeak = outofpeak.drop('UnderPeak', axis=0)
                    outofpeak.columns = ['mean']
                    outofpeak['width']=0.7
                    outofpeak['measure'] = outofpeak.index

                    fig = go.Figure()

                    fig.add_trace(go.Barpolar(
                        r=outofpeak['mean'],
                        theta=outofpeak['measure'],
                        width=outofpeak['width'],
                        base=0,
                        name='OutOfPeak',
                        marker_color='#FFFFFF',
                        marker_line_color='black',
                        marker_line_width=2,
                        opacity=1,
                    ))

                    fig.add_trace(go.Barpolar(
                          r=underpeak['mean'],
                          theta=underpeak['measure'],
                        width=underpeak['width'],
                          name='UnderPeak',
                        base=0,
                        marker_color='#BAB3BF',
                        marker_line_color='black',
                        marker_line_width=2,
                        opacity=1,
                    ))

                    fig.update_layout(
                        template='plotly_white',
                      polar=dict(
                          angularaxis_tickfont_size = 14,
                        radialaxis=dict(
                          visible=True,
                          range=[0, 1]
                        )),
                      showlegend=False
                    )

                    fig.write_image(outprefix + 'polar_ratings_sigdiff_{0}.svg'.format(parcel))
            except:
                print('no overlap between discovery and replication')

    #save results
    with open(outprefix + 'full_results_corrp{0}_prom{1}_width{2}.json'.format(round(bon_alpha,3), round(discbonmin,2), 5), 'w') as fp:
        json.dump(results, fp, indent=4)
        
    return(disc_sync_df, rep_sync_df)

In [ ]:
def find_peak_rating_diffs(peak_mask, ratings_file, video_dur, out_file, color='#BAB3BF', TR=TR, alpha=0.05, fdr=True, shift=6):
    '''
    Parameters
    ----------
    peak_mask: numpy ndarray
        timeseries mask in the shape of Nsamples, with 1 indicating peak and 0 indiciating nonpeak.
    ratings_file: filename
        CSV file containing the ratings to use to chracterize peak versus non-peak
    video_dur: float
        duration in seconds of the movie
    out_file: string
        Name to save the plot under
    TR = float
        Repetition Time in seconds
    alpha: float
        p-value to determine significance for t-tests (peak versus nonpeak)
    shift: int
        Number of samples to shift the ratings over to account for the delayed hemodynamic response
        
    Returns
    -------
    results: dict
        dictionary with full t-test results across all ratings
    
    '''
    
    results={'MeanRatings':dict(), 'Stats':dict(), 'pvals': dict()}

    ratings = pd.read_csv(ratings_file, index_col=0)
    ratings = ratings.iloc[0:int(video_dur/TR),:]
    ratings.loc[:,ratings.columns] = MinMaxScaler().fit_transform(ratings.to_numpy())
    ratings.index = range(6,int(video_dur/TR)+6)
    ratings = pd.concat([ratings, pd.DataFrame(np.nan, index=np.arange(0,6,1), columns=ratings.columns)])
    ratings = ratings.sort_index()
    ratings = ratings.iloc[0:int(video_dur/TR),:]

    for measure in ratings.columns:
        underpeak = ratings.loc[peak_mask==1, measure]
        outofpeak = ratings.loc[peak_mask==0, measure]
        underpeakmean = np.nanmean(underpeak)
        outofpeakmean = np.nanmean(outofpeak)
        rate_stat, rate_pval = scp.ttest_ind(underpeak, outofpeak, nan_policy='omit')
        results['Stats'][measure] = {'tstat': rate_stat, 'pval': rate_pval}
        results['pvals'][measure] = rate_pval
        results['MeanRatings'][measure] = {'underpeak': underpeakmean, 'outofpeak': outofpeakmean}

    # plot the differences
    if not fdr:
        sigratingsnames = [k for (k, v) in results['pvals'].items() if v<alpha]
    else: 
        from statsmodels.stats.multitest import multipletests
        pvals = list(results['pvals'].values())
        meas_names = list(results['pvals'].keys())
        sig, q, _, _ = multipletests(pvals, alpha, method='fdr_bh')
        sigratingsnames = [m for i,m in enumerate(meas_names) if q[i]<alpha]

    if (len(sigratingsnames) < 3) & (len(sigratingsnames) > 0):
        ratings_withpeakinfo = ratings
        ratings_withpeakinfo['UnderPeak'] = peak_mask
        temp = pd.melt(ratings_withpeakinfo, id_vars='UnderPeak', value_vars=sigratingsnames, value_name='Level',var_name='Rating')
        plt.figure(figsize=(2+2*len(sigratingsnames),6))
        f = sns.barplot(x='Rating', y='Level', hue='UnderPeak', data=temp, palette=['#FFFFFF', color], linewidth=2, edgecolor='k')
        f.legend_.remove()
        plt.tight_layout()
        plt.savefig(out_file)
        #plt.show()
        plt.close()
        ratings = ratings.drop('UnderPeak', axis=1)
    elif len(sigratingsnames) >= 3:
        if 'UnderPeak' in ratings.columns:
            ratings = ratings.drop('UnderPeak', axis=1)
        underpeak = ratings.loc[peak_mask==1, sigratingsnames].mean(axis=0).to_frame()
        if 'UnderPeak' in underpeak.index:
            underpeak = underpeak.drop('UnderPeak', axis=0)
        underpeak.columns = ['mean']
        underpeak['width']=0.4
        underpeak['measure'] = underpeak.index
        outofpeak = ratings.loc[peak_mask==0, sigratingsnames].mean(axis=0).to_frame()
        if 'UnderPeak' in outofpeak.columns:
            outofpeak = outofpeak.drop('UnderPeak', axis=0)
        outofpeak.columns = ['mean']
        outofpeak['width']=0.7
        outofpeak['measure'] = outofpeak.index

        fig = go.Figure()

        fig.add_trace(go.Barpolar(
            r=outofpeak['mean'],
            theta=outofpeak['measure'],
            width=outofpeak['width'],
            base=0,
            name='OutOfPeak',
            marker_color='#FFFFFF',
            marker_line_color='black',
            marker_line_width=2,
            opacity=1,
        ))

        fig.add_trace(go.Barpolar(
              r=underpeak['mean'],
              theta=underpeak['measure'],
            width=underpeak['width'],
              name='UnderPeak',
            base=0,
            marker_color=color,
            marker_line_color='black',
            marker_line_width=2,
            opacity=1,
        ))

        fig.update_layout(
            template='plotly_white',
            font=dict(family='Arial', size=12, color='black'),
          polar=dict(
              angularaxis_tickfont_size = 24,
            radialaxis=dict(
              visible=False,
              range=[0, 1]
            )),
          showlegend=False
        )

        fig.write_image(out_file)
    else:
        print('No differences between peak and nonpeak ratings')
        
    return(results)


def match_peak_to_clips(peaktimes, shift, video_file, outfolder, ratings_file, bool_list=None, dim_list=None):
    '''
    
    Parameters
    ----------
    peaktimes: DataFrame
        DataFrame containing and index of unique values (will be how clips are named) and a 'start' and 'end' column 
        with times in seconds.
    shift: float
        Time in seconds to shift the peak onset/offset by (is subtracted from each)
    video_file: str
        the moviepy compatible video file to pull clips from
    outfolder: str
        the folder path to save the clips under
    ratings_file: str
        CSV containing a pandas dataframe with an index of time in seconds and columns of video features to plot
    '''
    
    import moviepy.video.io.ffmpeg_tools as mpff
    from sklearn.preprocessing import MinMaxScaler
    features = pd.read_csv(ratings_file, index_col=0)
    features.loc[:,:] = MinMaxScaler().fit_transform(features.to_numpy())

    for clipnum in peaktimes.index:
        outfile = os.path.join(outfolder, 'clip{0}.mp4'.format(clipnum))
        start = peaktimes.loc[clipnum,'start'] - shift - 4
        end = peaktimes.loc[clipnum,'end'] - shift + 4
        if start<0:
            start=0
        if end>0:
            mpff.ffmpeg_extract_subclip(video_file, start, end, outfile)
        if not bool_list and not dim_list:
            fig, ax = plt.subplots(figsize=(6,4))
            features.loc[start:end,:].plot(kind='line', ax=ax)
            plt.tight_layout()
            sns.despine()
            plt.savefig(os.path.join(outfolder, 'clip{0}.svg'.format(clipnum)))
            plt.close()
        else:
            try:
                fig, ax = plt.subplots(figsize=(8,1.5*len(bool_list)))
                ax = features.loc[start:end,bool_list].plot(subplots=True, kind='area', ax=ax, xlim=(start,end), sharex=True, sharey=True)
                for a in range(0, len(bool_list)):
                    ax[a].axvline(x=peaktimes.loc[clipnum,'start'] - shift, color='k', linestyle='-')
                    ax[a].axvline(x=peaktimes.loc[clipnum,'end'] - shift, color='k', linestyle='-')
                sns.despine()
                plt.tight_layout()
                plt.savefig(os.path.join(outfolder, 'clip{0}_boolvarsleg.svg'.format(clipnum)))
                plt.close()
                fig, ax = plt.subplots(figsize=(8,1.5*len(bool_list)))
                ax = features.loc[start:end,bool_list].plot(subplots=True, kind='area', ax=ax, legend=False, xlim=(start,end), sharex=True, sharey=True)
                for a in range(0, len(bool_list)):
                    ax[a].axvline(x=peaktimes.loc[clipnum,'start'] - shift, color='k', linestyle='-')
                    ax[a].axvline(x=peaktimes.loc[clipnum,'end'] - shift, color='k', linestyle='-')
                sns.despine()
                plt.tight_layout()
                plt.savefig(os.path.join(outfolder, 'clip{0}_boolvars_noleg.svg'.format(clipnum)))
                plt.close()
            except:
                pass
            try:
                fig, ax = plt.subplots(figsize=(6,1.5*len(dim_list)))
                ax = features.loc[start:end,dim_list].plot(subplots=True, ax=ax, xlim=(start,end), sharex=True)
                for a in range(0, len(dim_list)):
                    ax[a].axvline(x=peaktimes.loc[clipnum,'start'] - shift, color='k', linestyle='-')
                    ax[a].axvline(x=peaktimes.loc[clipnum,'end'] - shift, color='k', linestyle='-')
                sns.despine()
                plt.tight_layout()
                plt.savefig(os.path.join(outfolder, 'clip{0}_dimvarsleg.svg'.format(clipnum)))
                plt.close()
                fig, ax = plt.subplots(figsize=(6,1.5*len(dim_list)))
                ax = features.loc[start:end,dim_list].plot(subplots=True, ax=ax, legend=False, xlim=(start,end), sharex=True)
                for a in range(0, len(dim_list)):
                    ax[a].axvline(x=peaktimes.loc[clipnum,'start'] - shift, color='k', linestyle='-')
                    ax[a].axvline(x=peaktimes.loc[clipnum,'end'] - shift, color='k', linestyle='-')
                sns.despine()
                plt.tight_layout()
                plt.savefig(os.path.join(outfolder, 'clip{0}_dimvars_nleg.svg'.format(clipnum)))
                plt.close()
            except:
                pass
            
            
def plot_network_activation(ts_data, nois, network_labels, time, groups, group_labels, outdir):
    
    colors = ['b','k','r','g','y']
    # convert to percent signal change
    ts_psc_data = (ts_data - ts_data.min(axis=0, keepdims=True)) / (ts_data.max(axis=0, keepdims=True) - ts_data.min(axis=0, keepdims=True))
    mean_sig = np.mean(ts_psc_data, axis=0, keepdims=True)
    ts_psc_data = ((ts_psc_data-mean_sig)/mean_sig)*100

    # average across network/region
    ts_psc_net_data = np.zeros((ts_psc_data.shape[0], len(nois), ts_psc_data.shape[2]))
    for i, n in enumerate(nois):
        ts_psc_net_data[:,i,:] = np.mean(ts_psc_data[:,network_labels==n,:], axis=1)

    # plot group level traces
    fig, ax = plt.subplots(len(nois),1,figsize=(12,2*len(nois)), sharex=True)
    for i, g in enumerate(groups):
        mean_sig = np.mean(ts_psc_net_data[:,:,group_labels==g], axis=2)
        for j, net in enumerate(nois):  
            ax[j].plot(time, mean_sig[:,j], color=colors[i], label=g)
            ax[j].set_xlim((0,time[-1]))
            ax[j].set_title(net)

    sns.despine()
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, 'network_activation_noleg.svg'))
    plt.legend()
    plt.savefig(os.path.join(outdir, 'network_activation_leg.svg'))
    plt.close()

## dynamic analysis

In [ ]:
for age in ['all','older','younger']:
    if age=='younger':
        clins = ['SCARED_P_SC']
    else:
        clins = ['SCARED_P_SC', 'SCARED_SR_SC']
    
    for clin in clins:
        if 'SR' in clin:
            other = 'MFQ_SR_Total'
        else:
            other = 'MFQ_P_Total'
        for sample in ['rubic','cbic']:
            for movie in ['TP','DM']:
                if age=='all':
                    sampleinfo = subinfo.loc[(subinfo['site']==sample) & (subinfo['movie']==movie) & np.isfinite(subinfo[clin]),:]
                    sampleinfo.loc[:,['age', 'female', 'meanFD', clin, other]] = \
                    IterativeImputer(random_state=42).fit_transform(sampleinfo.loc[:,['age', 'female', 'meanFD', clin, other]])
                    res = smf.ols('{0} ~ age + female + meanFD + {1}'.format(clin, other), data=sampleinfo).fit()
                    sampleinfo[clin] = res.resid.to_frame().iloc[:,0]
                else:
                    sampleinfo = subinfo.loc[(subinfo['age_group']==age) & (subinfo['site']==sample) & (subinfo['movie']==movie) & np.isfinite(subinfo[clin]),:]
                    sampleinfo.loc[:,['age', 'female', 'meanFD', clin, other]] = \
                    IterativeImputer(random_state=42).fit_transform(sampleinfo.loc[:,['age', 'female', 'meanFD', clin, other]])
                    res = smf.ols('{0} ~ female + meanFD + {1}'.format(clin, other), data=sampleinfo).fit()
                    sampleinfo[clin] = res.resid.to_frame().iloc[:,0]

                print(age, clin, movie, sample)
                bigdata_outfolder = os.path.join(big_data_dir, 'agegroup_similarity_regagesxs', age, 
                                      'dynamic_movie{0}_{1}'.format(movie, clin), 'peak_analysis_20')
                os.makedirs(bigdata_outfolder, exist_ok=True)
                out_folder = os.path.join(out_dir, 'agegroup_similarity_regagesxs', age, 
                                      'dynamic_movie{0}_{1}'.format(movie, clin), 'peak_analysis_20')
                os.makedirs(out_folder, exist_ok=True)
                group_data_file = os.path.join(out_folder, 'compiled_timeseries_data_{0}_movie{1}.npy'.format(sample, movie))
                if os.path.isfile(group_data_file):
                    group_data = np.load(group_data_file)
                else:
                    group_data = compile_ts_data(sampleinfo, movie, data_dir, group_data_file)

                # compute isps for upper and lower 20% scoring children
                top = (sampleinfo[clin]>=np.percentile(sampleinfo[clin], 80)).astype(int)
                bottom = (sampleinfo[clin]<=np.percentile(sampleinfo[clin], 20)).astype(int)

                phase_file = os.path.join(out_folder, 'upper_phase_data_{0}_movie{1}.npy'.format(sample, movie))
                if not os.path.isfile(phase_file):
                    phase_data = compute_group_phase(group_data[:,:, top==1], phase_file)
                else:
                    phase_data = np.load(phase_file)
                top_activation = group_data[:,:, top==1]
                np.save(os.path.join(out_folder, 'upper_signal_data_{0}_movie{1}.npy'.format(sample, movie)), top_activation)

                outprefix = os.path.join(bigdata_outfolder, 'upper_movie{0}_{1}_'.format(movie, sample))
                if not os.path.isfile(outprefix + 'isps_data.dat.npy'):
                    _, _ = compute_isps(phase_data, outprefix, small=True)

                phase_file = os.path.join(out_folder, 'lower_phase_data_{0}_movie{1}.npy'.format(sample, movie))
                if not os.path.isfile(phase_file):
                    phase_data = compute_group_phase(group_data[:,:, bottom==1], phase_file)
                else:
                    phase_data = np.load(phase_file)
                bottom_activation = group_data[:,:, bottom==1]
                np.save(os.path.join(out_folder, 'lower_signal_data_{0}_movie{1}.npy'.format(sample, movie)), bottom_activation)

                outprefix = os.path.join(bigdata_outfolder, 'lower_movie{0}_{1}_'.format(movie, sample))
                if not os.path.isfile(outprefix + 'isps_data.dat.npy'):
                    _, _ = compute_isps(phase_data, outprefix, small=True)


### what scenes elicit increased global synchrony in low/high symptom children?

In [ ]:
for age in ['all','younger','older']:
    if age=='younger':
        clins = ['SCARED_P_SC']
    else:
        clins = ['SCARED_P_SC', 'SCARED_SR_SC']
    
    for clin in clins:
        for movie in ['TP','DM']:
            print(age, clin, movie)
            if movie=='DM':
                video_file = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/HBN_video_coding/Videos/Despicable_Me_1000.mp4'
                movie_dur = 600
            else:
                video_file = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/HBN_video_coding/Videos/The_Present_0321.mp4'
                movie_dur = 200

            both_data = {'color':'gray','parcel_info':{}}
            time = np.arange(0,movie_dur, TR)
            print(time.shape)

            ratings_file = os.path.join(project_dir, 'HBN_video_coding/processing/summary/{0}_summary_codes_intuitivenames.csv'.format(movie))
            sig_parcs_file = os.path.join(out_dir, 'agegroup_similarity_regagesxs', age, 
                                          'ts_isc_movie{0}_{1}'.format(movie, clin), 'top_model_fits_replicable.dlabel.nii')
            sig_models = nib.load(sig_parcs_file).header.get_axis(0).label[0]
            model_labels = []
            mapping = [model_labels.append(a) for (a,b) in sig_models.values()]
            model_labels = model_labels[1:]
            model_labels = [a.split('_')[0] for a in model_labels]
            model_labels = np.array(model_labels)

            out_folder = os.path.join(out_dir, 'agegroup_similarity_regagesxs', age, 
                                      'dynamic_movie{0}_{1}'.format(movie, clin), 'peak_analysis_20')
            bigdata_outfolder = os.path.join(big_data_dir, 'agegroup_similarity_regagesxs', age, 
                                             'dynamic_movie{0}_{1}'.format(movie, clin), 'peak_analysis_20')

            for rel in ['lower','upper']:
                rel_folder = os.path.join(out_folder, 'global_comparison_ref-{0}'.format(rel))
                os.makedirs(rel_folder, exist_ok=True)
                if rel=='upper':
                    sigparcs = parcel_labels[model_labels=='AnnaKmin']
                    print(sigparcs)
                    siglabel = 'AnnaKmin'
                    disc_ref_isps = np.load(os.path.join(bigdata_outfolder, 'upper_movie{0}_rubic_isps_data.dat.npy'.format(movie)))
                    mask = np.tri(disc_ref_isps.shape[2], disc_ref_isps.shape[2], -1, dtype=int)
                    disc_ref_isps = disc_ref_isps[:,mask==1,:]
                    print(disc_ref_isps.shape)
                    disc_ref_signal = np.load(os.path.join(out_folder, 'upper_signal_data_rubic_movie{0}.npy'.format(movie)))

                    rep_ref_isps = np.load(os.path.join(bigdata_outfolder, 'upper_movie{0}_cbic_isps_data.dat.npy'.format(movie)))
                    mask = np.tri(rep_ref_isps.shape[2], rep_ref_isps.shape[2], -1, dtype=int)
                    rep_ref_isps = rep_ref_isps[:,mask==1,:]
                    rep_ref_signal = np.load(os.path.join(out_folder, 'upper_signal_data_cbic_movie{0}.npy'.format(movie)))
                    print(rep_ref_isps.shape)
                    
                    disc_comp_isps = np.load(os.path.join(bigdata_outfolder, 'lower_movie{0}_rubic_isps_data.dat.npy'.format(movie)))
                    mask = np.tri(disc_comp_isps.shape[2], disc_comp_isps.shape[2], -1, dtype=int)
                    disc_comp_isps = disc_comp_isps[:,mask==1,:]
                    disc_comp_signal = np.load(os.path.join(out_folder, 'lower_signal_data_rubic_movie{0}.npy'.format(movie)))
                    print(disc_comp_isps.shape)

                    rep_comp_isps = np.load(os.path.join(bigdata_outfolder, 'lower_movie{0}_cbic_isps_data.dat.npy'.format(movie)))
                    mask = np.tri(rep_comp_isps.shape[2], rep_comp_isps.shape[2], -1, dtype=int)
                    rep_comp_isps = rep_comp_isps[:,mask==1,:]
                    rep_comp_signal = np.load(os.path.join(out_folder, 'lower_signal_data_cbic_movie{0}.npy'.format(movie)))
                    print(rep_comp_isps.shape)

                elif rel=='lower':
                    sigparcs = parcel_labels[model_labels=='AnnaKmaxminmax']
                    print(sigparcs)
                    siglabel = 'AnnaKmaxminmax'
                    disc_comp_isps = np.load(os.path.join(bigdata_outfolder, 'upper_movie{0}_rubic_isps_data.dat.npy'.format(movie)))
                    mask = np.tri(disc_comp_isps.shape[2], disc_comp_isps.shape[2], -1, dtype=int)
                    disc_comp_isps = disc_comp_isps[:,mask==1,:]
                    disc_comp_signal = np.load(os.path.join(out_folder, 'upper_signal_data_rubic_movie{0}.npy'.format(movie)))

                    rep_comp_isps = np.load(os.path.join(bigdata_outfolder, 'upper_movie{0}_cbic_isps_data.dat.npy'.format(movie)))
                    mask = np.tri(rep_comp_isps.shape[2], rep_comp_isps.shape[2], -1, dtype=int)
                    rep_comp_isps = rep_comp_isps[:,mask==1,:]
                    rep_comp_signal = np.load(os.path.join(out_folder, 'upper_signal_data_cbic_movie{0}.npy'.format(movie)))

                    disc_ref_isps = np.load(os.path.join(bigdata_outfolder, 'lower_movie{0}_rubic_isps_data.dat.npy'.format(movie)))
                    mask = np.tri(disc_ref_isps.shape[2], disc_ref_isps.shape[2], -1, dtype=int)
                    disc_ref_isps = disc_ref_isps[:,mask==1,:]
                    disc_ref_signal = np.load(os.path.join(out_folder, 'lower_signal_data_rubic_movie{0}.npy'.format(movie)))

                    rep_ref_isps = np.load(os.path.join(bigdata_outfolder, 'lower_movie{0}_cbic_isps_data.dat.npy'.format(movie)))
                    mask = np.tri(rep_ref_isps.shape[2], rep_ref_isps.shape[2], -1, dtype=int)
                    rep_ref_isps = rep_ref_isps[:,mask==1,:]
                    rep_ref_signal = np.load(os.path.join(out_folder, 'lower_signal_data_rubic_movie{0}.npy'.format(movie)))

                if len(sigparcs)==0:
                    print('no sig parcels found', clin, movie, rel)
                    print(sigparcs)
                    break

                sig_rho = nib.load(os.path.join(out_dir, 'agegroup_similarity_regagesxs', age, 'ts_isc_movie{0}_{1}'.format(movie,clin), 
                                                'movie{0}_isc_{1}_{2}_maskedrho_fdr0.01127.pscalar.nii'.format(movie, clin, siglabel))).get_fdata()
                sig_rho_parcels = parcel_labels[np.isfinite(np.squeeze(sig_rho))]

                disc_ref_isps = np.mean(disc_ref_isps[model_labels==siglabel,:,:], axis=0)
                rep_ref_isps = np.mean(rep_ref_isps[model_labels==siglabel,:,:], axis=0)
                disc_comp_isps = np.mean(disc_comp_isps[model_labels==siglabel,:,:], axis=0)
                rep_comp_isps = np.mean(rep_comp_isps[model_labels==siglabel,:,:], axis=0)

                peaks = pd.DataFrame(0,index=range(0,len(time)), columns=['disc','rep'])
                peaks['time'] = time
                
                # Create permuted T distribution
                perm_dist_file = os.path.join(bigdata_outfolder, 'perm_tdist.npy')
                if not os.path.isfile(perm_dist_file):
                    null_dist = np.empty(10000)
                    n_ref = disc_ref_isps.shape[0]
                    perm_data = np.concatenate([disc_ref_isps,disc_comp_isps], axis=0)
                    perm_data_shape = perm_data.shape
                    for i in range(0,10000):
                        perm_data = perm_data.flatten()
                        np.random.shuffle(perm_data)
                        perm_data = np.reshape(perm_data, perm_data_shape)
                        null_dist[i], _ = scp.ttest_ind(np.squeeze(perm_data[:n_ref,0]), np.squeeze(perm_data[n_ref:,0]), axis=None, 
                                                        alternative='greater', equal_var=False)
                    np.save(perm_dist_file, null_dist)
                else:
                    null_dist = np.load(perm_dist_file)

                # rolling t-test for discovery sample
                for t in range(0, disc_ref_isps.shape[1]):
                    tstat, _ = scp.ttest_ind(np.squeeze(disc_ref_isps[:,t]), np.squeeze(disc_comp_isps[:,t]), axis=None, 
                                             alternative='greater', equal_var=False)
                    peaks.loc[t,'disc_tstat'] = tstat
                    p = (np.sum((null_dist>=tstat).astype(int)) + 1) / 10001

                    if p<0.05:
                        peaks.loc[t,'disc'] = 1
                    else:
                        peaks.loc[t,'disc'] = 0

                # rolling t-test for replication sample
                for t in range(0, disc_ref_isps.shape[1]):
                    tstat, _ = scp.ttest_ind(np.squeeze(rep_ref_isps[:,t]), np.squeeze(rep_comp_isps[:,t]), axis=None, 
                                             alternative='greater', equal_var=False)
                    p = (np.sum((null_dist>=tstat).astype(int)) + 1) / 10001
                    peaks.loc[t,'rep_tstat'] = tstat
                    if p<0.05:
                        peaks.loc[t,'rep'] = 1
                    else:
                        peaks.loc[t,'rep'] = 0

                # organize and clean peaks
                both_data['global']={'long_peaks':peaks}
                both_data['global']['mean_ref_disc'] = np.mean(disc_ref_isps, axis=0)
                both_data['global']['mean_ref_rep'] = np.mean(rep_ref_isps, axis=0)
                both_data['global']['mean_comp_disc'] = np.mean(disc_comp_isps, axis=0)
                both_data['global']['mean_comp_rep'] = np.mean(rep_comp_isps, axis=0)
                both_data['global']['long_peaks'].index = time
                peaks['both'] = ((peaks['disc']==1) & (peaks['rep']==1)).astype(int)
                peaks['segment'] = (peaks['both'].diff(1) != 0).astype(int).cumsum()
                res = pd.DataFrame({'start': peaks.groupby('segment').time.first(),
                                    'end': peaks.groupby('segment').time.last(),
                                    'dur': peaks.groupby('segment').time.last()-peaks.groupby('segment').time.first(),
                                    'disc_mean':peaks.groupby('segment')['disc_tstat'].mean(),
                                    'rep_mean':peaks.groupby('segment')['rep_tstat'].mean(),
                                    'disc_max':peaks.groupby('segment')['disc_tstat'].max(),
                                    'rep_max':peaks.groupby('segment')['rep_tstat'].max(),
                                    'value': peaks.groupby('segment')['both'].mean()}).reset_index(drop=True)
                peaks_to_delete = res.loc[(res['value'] == 1) & (res['dur']<4), :]
                if len(peaks_to_delete)>0:
                    for a in peaks_to_delete.index:
                        both_data['global']['long_peaks'].loc[peaks_to_delete.loc[a,'start']:peaks_to_delete.loc[a,'end'], 'both'] = 0
                peaks = res.loc[(res['value'] == 1) & (res['dur']>=4), :]

                # plot discovery and replication
                fig, ax = plt.subplots(2,1,figsize=(12,6), sharex=True, sharey=True)
                for i_c, c in enumerate(peaks['start']):
                    ax[0].axvspan(peaks['start'].to_numpy()[i_c], peaks['end'].to_numpy()[i_c], color='#9A38E7',fill=True, alpha=0.4)
                    ax[1].axvspan(peaks['start'].to_numpy()[i_c], peaks['end'].to_numpy()[i_c], color='#9A38E7',fill=True, alpha=0.4)

                ax[0].plot(time, np.squeeze(np.mean(disc_comp_isps, axis=0)), color='k')
                ax[0].plot(time, np.squeeze(np.mean(disc_ref_isps, axis=0)), color='#9A38E7')
                ax[0].set_xlim((0,movie_dur))
                ax[0].set_title('Discovery')            
                ax[1].plot(time, np.squeeze(np.mean(rep_comp_isps, axis=0)), color='k')
                ax[1].plot(time, np.squeeze(np.mean(rep_ref_isps, axis=0)), color='#9A38E7')
                ax[1].set_xlim((0,movie_dur))
                ax[1].set_title('Replication')

                both_data['global']['peaks'] = peaks
                plt.suptitle('global')
                sns.despine()
                plt.tight_layout()
                plt.savefig(os.path.join(rel_folder, 'bysite_synchrony_{0}_{1}.svg'.format('global', rel)))
                plt.close()

                # plot discovery and replication tstats
                fig, ax = plt.subplots(2,1,figsize=(12,6), sharex=True, sharey=True)
                for i_c, c in enumerate(peaks['start']):
                    ax[0].axvspan(peaks['start'].to_numpy()[i_c], peaks['end'].to_numpy()[i_c], color=both_data['color'],fill=True, alpha=0.4)
                    ax[1].axvspan(peaks['start'].to_numpy()[i_c], peaks['end'].to_numpy()[i_c], color=both_data['color'],fill=True, alpha=0.4)

                ax[0].plot(time, both_data['global']['long_peaks']['disc_tstat'], color='k')
                ax[0].set_xlim((0,movie_dur))
                ax[0].set_title('Discovery')            
                ax[1].plot(time, both_data['global']['long_peaks']['rep_tstat'], color='k')
                ax[1].set_xlim((0,movie_dur))
                ax[1].set_title('Replication')

                both_data['global']['peaks'] = peaks
                plt.suptitle('global')
                sns.despine()
                plt.tight_layout()
                plt.savefig(os.path.join(rel_folder, 'Network_bysite_tstat_{0}_{1}.svg'.format('global', rel)))
                plt.close()

                # plot just discovery
                fig, ax = plt.subplots(1,1,figsize=(16,2.5), sharex=True)
                peaks = both_data['global']['peaks']
                for i_c, c in enumerate(peaks['start']):
                    ax.axvspan(peaks['start'].to_numpy()[i_c], peaks['end'].to_numpy()[i_c], color=both_data['color'],fill=True, alpha=0.4)
                    ax.axvspan(peaks['start'].to_numpy()[i_c], peaks['end'].to_numpy()[i_c], color=both_data['color'],fill=True, alpha=0.4)

                ax.plot(time, np.squeeze(np.mean(disc_comp_isps, axis=0)), color='k')
                ax.plot(time, np.squeeze(np.mean(disc_ref_isps, axis=0)), color='#9A38E7')
                ax.set_xlim((0,movie_dur))
                ax.set_xlabel('Time (s)')
                ax.set_ylabel('Synchrony')
                sns.despine()
                plt.tight_layout()
                plt.savefig(os.path.join(rel_folder, 'Networks_all_synchrony_{0}20.svg'.format(rel)))
                plt.close()

                ### Find differences in peak versus non-peak in terms of video features
                peak_mask = both_data['global']['long_peaks']['both'].to_numpy()
                ratings_file = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/HBN_video_coding/processing/v1/summary/{0}_summary_codes_intuitivenames.csv'.format(movie)
                out_file = os.path.join(rel_folder, '{0}_peak_differences_{1}.svg'.format('global',rel))
                if (peak_mask.max()==1):
                    both_data['global']['peak_quant_analysis'] = find_peak_rating_diffs(peak_mask, ratings_file, movie_dur, out_file, 
                                                                                       color=both_data['color'], TR=TR, alpha=0.05, fdr=True, shift=6)

                ### find which clips are promoting the most activation
                peaktimes = both_data['global']['peaks']
                peaktimes.loc[:,'meanact'] = peaktimes.loc[:,['disc_mean','rep_mean']].mean(axis=1)
                peaktimes.loc[:,'maxact'] = peaktimes.loc[:,['disc_max','rep_max']].mean(axis=1)
                peaktimes = peaktimes.sort_values(by='maxact', ascending=False)
                peaktimes.index = range(1, len(peaktimes)+1)
                both_data['global']['peak_qual_analysis'] = peaktimes
                outfolder = os.path.join(rel_folder, 'clips_{0}_{1}'.format('global',rel))
                os.makedirs(outfolder, exist_ok=True)
                peaktimes.to_csv(os.path.join(outfolder, 'peak_analysis_{0}_{1}.csv'.format('global',rel)))
                shift = 5 #in seconds
                ratings_file = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/HBN_video_coding/processing/v1/summary/{0}_summary_codes10Hz_intuitivenames.csv'.format(movie)
                match_peak_to_clips(peaktimes, shift, video_file, outfolder, ratings_file, 
                                    bool_list=['faces','body','closeup','spoken_words','written_words'],
                                    dim_list=['positive','negative','brightness','loudness','saliency'])

                ### create activation maps
                for a in peaktimes.index.to_list():
                    start = int(peaktimes.loc[a,'start']/TR)
                    end = int(peaktimes.loc[a, 'end']/TR)
                    mean_sig_disc_ref = np.full((1, parcel_labels.shape[0]), np.nan)
                    mean_sig_disc_comp = np.full((1, parcel_labels.shape[0]), np.nan)
                    mean_sig_rep_ref = np.full((1, parcel_labels.shape[0]), np.nan)
                    mean_sig_rep_comp = np.full((1, parcel_labels.shape[0]), np.nan)
                    activation_disc_tstat = np.full((1, parcel_labels.shape[0]), np.nan)
                    activation_disc_pval = np.full((1, parcel_labels.shape[0]), np.nan)
                    activation_rep_tstat = np.full((1, parcel_labels.shape[0]), np.nan)
                    activation_rep_pval = np.full((1, parcel_labels.shape[0]), np.nan)

                    for parc in sig_rho_parcels:
                        parcind = np.where(parcel_labels==parc)[0][0]
                        # compute mean activation for each segment
                        mean_sig_disc_ref[0, parcind] = np.mean(disc_ref_signal[start:end, parcind,:], axis=None)
                        mean_sig_rep_ref[0, parcind] = np.mean(rep_ref_signal[start:end, parcind,:], axis=None)
                        mean_sig_disc_comp[0, parcind] = np.mean(disc_comp_signal[start:end, parcind,:], axis=None)
                        mean_sig_rep_comp[0, parcind] = np.mean(rep_comp_signal[start:end, parcind,:], axis=None)

                        # Compute t-stat for each parcel and each segment
                        tstat, pval = scp.ttest_ind(np.squeeze(np.mean(disc_ref_signal[start:end, parcind,:], axis=0)), 
                                                    np.squeeze(np.mean(disc_comp_signal[start:end, parcind,:], axis=0)), 
                                                    axis=None, alternative='two-sided')
                        activation_disc_tstat[0, parcind] = tstat
                        activation_disc_pval[0, parcind] = pval
                        del tstat, pval
                        tstat, pval = scp.ttest_ind(np.squeeze(np.mean(rep_ref_signal[start:end, parcind,:], axis=0)), 
                                                    np.squeeze(np.mean(rep_comp_signal[start:end, parcind,:], axis=0)), 
                                                    axis=None, alternative='two-sided')

                        activation_rep_tstat[0, parcind] = tstat
                        activation_rep_pval[0, parcind] = pval
                        del tstat, pval, parcind

                    # save as ciftis
                    ciftilabels = ['mean_activation_ref_disc','mean_activation_comp_disc','mean_activation_ref_rep','mean_activation_comp_rep', 
                                   'activation_disc_tstat', 'activation_disc_pval', 'activation_rep_tstat', 'activation_rep_pval']
                    ciftidata = [mean_sig_disc_ref, mean_sig_disc_comp, mean_sig_rep_ref, mean_sig_rep_comp, activation_disc_tstat, 
                                 activation_disc_pval, activation_rep_tstat, activation_rep_pval]
                    for i, c in enumerate(ciftilabels):
                        img = nib.cifti2.cifti2.Cifti2Image(ciftidata[i], (ax0, ax1))
                        nib.save(img, os.path.join(outfolder, 'clip{0}_{1}.pscalar.nii'.format(a, c)))

                ## pickle results
                f = open(os.path.join(rel_folder, 'final_peaks_data_{0}.pkl'.format(rel)),'wb')
                pickle.dump(both_data,f)